<a href="https://colab.research.google.com/github/ZhenwenZhang1108/ADL3_PartB/blob/main/PartB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install torcheval

In [22]:
import sklearn
from sklearn.metrics import f1_score
import torch
import torch.nn as nn
import torch.utils
import torch.optim as optim
import torch
from torch.utils.data import Dataset,DataLoader,random_split
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
from torch.nn import functional as F
from torchvision.datasets.utils import download_url
from torch import Tensor
from sklearn.preprocessing import OneHotEncoder
from torcheval.metrics.functional import multiclass_f1_score
from google.colab import drive
from torcheval.metrics.functional.classification import binary_recall,multiclass_recall
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import os
path = "/content/drive/MyDrive/adl3/"
os.chdir(path)
print(os.listdir())

['model_lstm1.pth']


In [24]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


创建数据库(list存两个各自的数组？一个数组里装x,另一个装y?，还是必须构建x到y的一一对应关系)。
为每段序列添加一个标签（打标签的规则是什么？总不能随机打标签吧）。

对序列进行编码，abc的话采用One-hot编码应该可以。

对序列进行编码，只有abc三个字母的话采用One-hot编码应该可以。是否需要补齐<padding>？

如果我们设置batch size=1，那就不需要padding了？？？
但如此设置，恐怕挨个挪到GPU上就会很慢了，可能就得在CPU上直接训练？

In [25]:

#自定义一个Dataset类，必需包含init, len, getitem
class CustomDataset(Dataset):
    def __init__(self):
        self.data = []
        self.targets = []

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index], self.targets[index]

    def append(self, data, target):
        self.data.append(data)
        self.targets.append(target)

# 创建自定义数据集对象
dataset = CustomDataset()
sum = 0
k=0
# 逐个添加数据和标签
for x in range(1,19):
  for y in range(1,19):
    for z in range(1,19):
     if x+y+z>20:
       continue
     input_string = 'a'*x + 'b'*y + 'c'*z
     sum+=1
     input_array = np.array(list(input_string)).reshape(-1, 1)
     #创建并适配 OneHotEncoder
     encoder = OneHotEncoder(sparse_output=False)
     data = torch.FloatTensor(encoder.fit_transform(input_array))
     if x==y or y==z or x==z :
      target=torch.tensor(1)
      k+=1
     else:
      target=torch.tensor(0)
     dataset.append(data, target)

batch_size = 1
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

print(next(iter(dataloader))[1])
print(k)

tensor([1])
258


In [26]:
print(dataset[11][0].shape)
print(dataset[12][0].shape)
print(dataset[13][0].shape)

torch.Size([14, 3])
torch.Size([15, 3])
torch.Size([16, 3])


In [27]:
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0
        for texts, labels in train_loader:
            # 前向传播
            outputs = model(texts)
            loss = criterion(outputs, labels)

            # 反向传播和优化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

In [28]:
def eval_model(model,test_loader):
    model.eval()
    input = torch.tensor([])
    target = torch.tensor([])
    with torch.no_grad():
      for texts, labels in test_loader:
          outputs = model(texts)
          predicted = torch.argmax(outputs,1)
          input = torch.cat((input,predicted))
          target = torch.cat((target,labels))
    #F1score还没算呢！！！
    input = input.numpy()
    target = target.numpy()
    accuracy = 100 * (input == target).sum() / len(target)
   ## recall = binary_recall(input, target)
    f1score = f1_score(y_pred=input,y_true=target)
    print(f'Validation Accuracy: {accuracy:.2f}%')
    print(f'Validation F1 score: {f1score:.2f}')
    print(input.shape)
    print(target.shape)

RNN模型建立
https://pytorch.org/docs/stable/generated/torch.nn.RNN.html

In [29]:
class RNNClassifier(nn.Module):
    def __init__(self, nlayers=1, embed_size=3, hidsize=10, bidirect=False, output_size=2):
        super(RNNClassifier, self).__init__()
        #one-hot编码，字母本身没有额外信息需要学习，所以不需要embedding层??
        self.embed = nn.Linear(in_features=embed_size,out_features=hidsize)
        self.rnn = nn.RNN(input_size=hidsize, num_layers=nlayers, hidden_size=hidsize, bidirectional=bidirect, batch_first=True)
        if bidirect==True:
          self.bi=2
        else:
          self.bi=1
        self.fc = nn.Linear(hidsize*self.bi*nlayers,output_size)
    def forward(self, x):
        x= self.embed(x)
        output,h_n = self.rnn(input=x)
        h_n = h_n.permute(1,0,2) #这里要小心，即使开了batch_first=True,第一个维度上也不是batch_size，这是pytorch.RNN的设定
        h_n = torch.flatten(h_n,start_dim=-2)
        out = self.fc(h_n)
        return out #注意这里的output没做softmax

In [33]:
model_rnn = RNNClassifier(hidsize=100, bidirect=True).to(device)
criterion_CE = nn.CrossEntropyLoss() #自带softmax
optimizer_adam = optim.Adam(model_rnn.parameters(), lr=0.001)

x = torch.rand(6,11,3) #(batch_size,seq_len,embed_size)
y = model_rnn(x)
z = torch.argmax(y,1)
print(z.shape)

torch.Size([6])


In [34]:
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

trainloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

print(len(trainloader))
print(len(testloader))

798
342


In [35]:
# 训练模型
train_model(model=model_rnn, train_loader=trainloader, criterion=criterion_CE, optimizer = optimizer_adam, num_epochs=20)
# 测试模型


Epoch [1/20], Loss: 0.5743
Epoch [2/20], Loss: 0.5297
Epoch [3/20], Loss: 0.5201
Epoch [4/20], Loss: 0.5081
Epoch [5/20], Loss: 0.5037
Epoch [6/20], Loss: 0.5061
Epoch [7/20], Loss: 0.5096
Epoch [8/20], Loss: 0.4979
Epoch [9/20], Loss: 0.5164
Epoch [10/20], Loss: 0.4931
Epoch [11/20], Loss: 0.5244
Epoch [12/20], Loss: 0.5320
Epoch [13/20], Loss: 0.5181
Epoch [14/20], Loss: 0.4710
Epoch [15/20], Loss: 0.4809
Epoch [16/20], Loss: 0.4604
Epoch [17/20], Loss: 0.4311
Epoch [18/20], Loss: 0.4221
Epoch [19/20], Loss: 0.4227
Epoch [20/20], Loss: 0.4238


In [37]:
eval_model(model_rnn,testloader)
torch.save(model_rnn.state_dict(), './model_rnn1.pth')

Validation Accuracy: 83.33%
Validation F1 score: 0.45
(342,)
(342,)


LSTM模型
https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html

In [ ]:
class LSTMClassifier(nn.Module):
    def __init__(self, nlayers=1, embed_size=3, hidsize=100, bidirect=False, output_size=2):
        super(LSTMClassifier, self).__init__()
        self.embed = nn.Linear(in_features=embed_size,out_features=hidsize)
        self.lstm = nn.LSTM(input_size=hidsize, num_layers=nlayers, hidden_size=hidsize, bidirectional=bidirect, batch_first=True)
        if bidirect==True:
          self.bi=2
        else:
          self.bi=1
        self.fc = nn.Linear(hidsize*self.bi*nlayers,output_size)
    def forward(self, x):
        x = self.embed(x)
        output, (h_n, c_n) = self.lstm(input=x)
        h_n = h_n.permute(1,0,2) #这里要小心，即使开了batch_first=True,第一个维度上也不是batch_size，这是pytorch.RNN的设定
        h_n = torch.flatten(h_n,start_dim=-2)
        out = self.fc(h_n)
        return out #注意这里的output没做softmax

In [ ]:
model_lstm = LSTMClassifier().to(device)
criterion_CE = nn.CrossEntropyLoss() #自带softmax
optimizer_adam = optim.Adam(model_lstm.parameters(), lr=0.001)

x = torch.rand(6,11,3) #(batch_size,seq_len,embed_size)
y = model_lstm(x)
z = torch.argmax(y,1)
print(z.shape)

torch.Size([6])


In [ ]:
train_model(model=model_lstm, train_loader=trainloader, criterion=criterion_CE, optimizer = optimizer_adam, num_epochs=20)
# 测试模型

Epoch [1/20], Loss: 0.5399
Epoch [2/20], Loss: 0.5217
Epoch [3/20], Loss: 0.5114
Epoch [4/20], Loss: 0.4941
Epoch [5/20], Loss: 0.4876
Epoch [6/20], Loss: 0.4608
Epoch [7/20], Loss: 0.4437
Epoch [8/20], Loss: 0.3989
Epoch [9/20], Loss: 0.3743
Epoch [10/20], Loss: 0.3530
Epoch [11/20], Loss: 0.3404
Epoch [12/20], Loss: 0.3183
Epoch [13/20], Loss: 0.2957
Epoch [14/20], Loss: 0.2517
Epoch [15/20], Loss: 0.1719
Epoch [16/20], Loss: 0.1429
Epoch [17/20], Loss: 0.0928
Epoch [18/20], Loss: 0.0856
Epoch [19/20], Loss: 0.0330
Epoch [20/20], Loss: 0.0346


In [ ]:
eval_model(model_lstm,testloader)

Validation Accuracy: 98.54%
Validation F1 score: 0.97
(342,)
(342,)


In [ ]:
torch.save(model_lstm.state_dict(), './model_lstm1.pth')

泛化能力评估：长度21-50

参数优化：是否进行embedding; hidden_size取50，75，100.。。。。